In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
project_id = 'test-project-221821'

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.environ['HOME'], 'gcp/secret/admin-service-account.json')
import sys
sys.path.append(os.path.join(os.environ['HOME'], 'gcp'))

from dataproc.client import DataProc

In [4]:
region = 'europe-west1'
zone_letter = 'd'

dp = DataProc(project_id, region, zone_letter)

# Create a cluster

In [6]:
cluster_name = 'cluster-test'
master_machine_type = 'n1-standard-1'
nr_masters = 1
master_boot_disk_gb = 200
worker_machine_type = 'n1-standard-1'
nr_workers = 2
worker_boot_disk_gb = 100
metadata = {'MINICONDA_VARIANT': '3', 'MINICONDA_VERSION': '4.5.11'}
initialization_file = 'gs://letter_counts_example/initialize_cluster.sh'

dp.create_cluster(cluster_name,
                  master_machine_type,
                  nr_masters,
                  master_boot_disk_gb,
                  worker_machine_type,
                  nr_workers,
                  worker_boot_disk_gb,
                  metadata,
                  initialization_file)

{'name': 'projects/test-project-221821/regions/europe-west1/operations/f13830f5-2504-4b34-a461-3694d1e1bb57',
 'metadata': {'@type': 'type.googleapis.com/google.cloud.dataproc.v1.ClusterOperationMetadata',
  'clusterName': 'cluster-test',
  'clusterUuid': '9bc22874-4a27-4176-9e8e-e0e3d005fb67',
  'status': {'state': 'PENDING',
   'innerState': 'PENDING',
   'stateStartTime': '2019-01-19T22:00:12.586Z'},
  'operationType': 'CREATE',
  'description': 'Create cluster with 2 workers'}}

# Submit a pyspark job

In [11]:
main_python_file = 'gs://letter_counts_example/calculate_letter_statistics.py'
script_parameters = ['--project_id={}'.format(project_id),
                     '--input_file_glob=gs://letter_counts_example/letter_files/inputs/*',
                     '--output_path=gs://letter_counts_example/letter_files/outputs/letter_statistics.csv']

dp.submit_pyspark_job(cluster_name, main_python_file, script_parameters)

'45e9ed6a-3346-4ddc-a384-da9e86ab28c3'

# Cleanup: delete cluster

In [12]:
dp.delete_cluster(cluster_name)

{'name': 'projects/test-project-221821/regions/europe-west1/operations/c125d49a-c5c8-48c7-9d17-ca3173e5aa5e',
 'metadata': {'@type': 'type.googleapis.com/google.cloud.dataproc.v1.ClusterOperationMetadata',
  'clusterName': 'cluster-test',
  'clusterUuid': '9bc22874-4a27-4176-9e8e-e0e3d005fb67',
  'status': {'state': 'PENDING',
   'innerState': 'PENDING',
   'stateStartTime': '2019-01-19T22:14:55.111Z'},
  'operationType': 'DELETE',
  'description': 'Delete cluster'}}